In [1]:
# Derived from keras-rl
import opensim as osim
import numpy as np
import sys

from keras.models import Sequential, Model
from keras.layers import Dense, Activation, Flatten, Input, concatenate
from keras.optimizers import Adam

import numpy as np

from rl.agents import DDPGAgent
from rl.agents import NAFAgent
from rl.memory import SequentialMemory
from rl.random import OrnsteinUhlenbeckProcess

from osim.env import *
from osim.http.client import Client

from keras.optimizers import RMSprop

import argparse
import math

Using Theano backend.


In [3]:
# Command line parameters
#parser = argparse.ArgumentParser(description='Train or test neural net motor controller')
#parser.add_argument('--train', dest='train', action='store_true', default=True)
#parser.add_argument('--test', dest='train', action='store_false', default=True)
#parser.add_argument('--steps', dest='steps', action='store', default=10000, type=int)
#parser.add_argument('--visualize', dest='visualize', action='store_true', default=False)
#parser.add_argument('--model', dest='model', action='store', default="example.h5f")
#parser.add_argument('--token', dest='token', action='store', required=False)
#args = parser.parse_args()
visualize = False
steps = 500000
train = True
model = "sample1129_CDQN"

In [4]:
# Load walking environment
env = RunEnv(visualize)
env.reset()

nb_actions = env.action_space.shape[0]

# Total number of steps in training
nallsteps = steps

# Create networks for DDPG
# Build all necessary models: V, mu, and L networks.
V_model = Sequential()
V_model.add(Flatten(input_shape=(1,) + env.observation_space.shape))
V_model.add(Dense(32))
V_model.add(Activation('relu'))
V_model.add(Dense(32))
V_model.add(Activation('relu'))
V_model.add(Dense(16))
V_model.add(Activation('relu'))
V_model.add(Dense(1))
V_model.add(Activation('linear'))
print(V_model.summary())

mu_model = Sequential()
mu_model.add(Flatten(input_shape=(1,) + env.observation_space.shape))
mu_model.add(Dense(32))
mu_model.add(Activation('relu'))
mu_model.add(Dense(32))
mu_model.add(Activation('relu'))
mu_model.add(Dense(16))
mu_model.add(Activation('relu'))
mu_model.add(Dense(nb_actions))
mu_model.add(Activation('sigmoid'))
print(mu_model.summary())

action_input = Input(shape=(nb_actions,), name='action_input')
observation_input = Input(shape=(1,) + env.observation_space.shape, name='observation_input')
x = concatenate([action_input, Flatten()(observation_input)])
x = Dense(64)(x)
x = Activation('relu')(x)
x = Dense(64)(x)
x = Activation('relu')(x)
x = Dense(((nb_actions * nb_actions + nb_actions) // 2))(x)
x = Activation('linear')(x)
L_model = Model(input=[action_input, observation_input], output=x)
print(L_model.summary())

# Set up the agent for training
memory = SequentialMemory(limit=100000, window_length=1)
random_process = OrnsteinUhlenbeckProcess(theta=.15, mu=0., sigma=.2, size=env.noutput)
#agent = DDPGAgent(nb_actions=nb_actions, actor=actor, critic=critic, critic_action_input=action_input,
#                 memory=memory, nb_steps_warmup_critic=100, nb_steps_warmup_actor=100,
#                  random_process=random_process, gamma=.99, target_model_update=1e-3,
#                  delta_clip=1.)
agent = NAFAgent(nb_actions=env.noutput, V_model=V_model, L_model=L_model, mu_model=mu_model,
                           memory=memory, nb_steps_warmup=1000, random_process=random_process,
                           gamma=.99, target_model_update=0.1)
agent.compile(Adam(lr=.001, clipnorm=1.), metrics=['mae'])

# Okay, now it's time to learn something! We visualize the training here for show, but this
# slows down training quite a lot. You can always safely abort the training prematurely using
# Ctrl + C.
if train:
    agent.fit(env, nb_steps=nallsteps, visualize=False, verbose=1, nb_max_episode_steps=env.timestep_limit, log_interval=10000)
    # After training is done, we save the final weights.
    agent.save_weights(model, overwrite=True)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_1 (Flatten)          (None, 41)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 32)                1344      
_________________________________________________________________
activation_1 (Activation)    (None, 32)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 32)                1056      
_________________________________________________________________
activation_2 (Activation)    (None, 32)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 16)                528       
_________________________________________________________________
activation_3 (Activation)    (None, 16)                0         
__________

C:\Users\caire\Anaconda2\envs\opensim-rl\lib\site-packages\ipykernel_launcher.py:45: UserWarning: Update your `Model` call to the Keras 2 API: `Model(outputs=Elemwise{i..., inputs=[/action_i...)`


____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
observation_input (InputLayer)   (None, 1, 41L)        0                                            
____________________________________________________________________________________________________
action_input (InputLayer)        (None, 18L)           0                                            
____________________________________________________________________________________________________
flatten_3 (Flatten)              (None, 41)            0           observation_input[0][0]          
____________________________________________________________________________________________________
concatenate_1 (Concatenate)      (None, 59)            0           action_input[0][0]               
                                                                   flatten_3[0][0]         

WARNING (theano.tensor.blas): We did not found a dynamic library into the library_dir of the library we use for blas. If you use ATLAS, make sure to compile it with dynamics library.


10000/10000 [==============================] - 590s - reward: -0.0066   
83 episodes - episode_reward: -0.801 [-0.821, -0.786] - loss: 0.001 - mean_absolute_error: 0.024 - mean_q: 1.419

Interval 2 (10000 steps performed)
10000/10000 [==============================] - 589s - reward: -0.0067   
83 episodes - episode_reward: -0.802 [-0.823, -0.786] - loss: 0.000 - mean_absolute_error: 0.006 - mean_q: -0.197

Interval 3 (20000 steps performed)
10000/10000 [==============================] - 586s - reward: -0.0066   
82 episodes - episode_reward: -0.803 [-0.842, -0.786] - loss: 0.000 - mean_absolute_error: 0.004 - mean_q: -0.324

Interval 4 (30000 steps performed)
10000/10000 [==============================] - 585s - reward: -0.0066   
82 episodes - episode_reward: -0.806 [-0.824, -0.786] - loss: 0.000 - mean_absolute_error: 0.004 - mean_q: -0.301

Interval 5 (40000 steps performed)
10000/10000 [==============================] - 602s - reward: -0.0066   
82 episodes - episode_reward: -0.807

10000/10000 [==============================] - 978s - reward: 0.0062   
92 episodes - episode_reward: 0.674 [0.447, 0.730] - loss: 0.005 - mean_absolute_error: 0.025 - mean_q: 1.192

Interval 39 (380000 steps performed)
10000/10000 [==============================] - 987s - reward: 0.0064   
96 episodes - episode_reward: 0.662 [0.456, 0.887] - loss: 0.012 - mean_absolute_error: 0.046 - mean_q: 2.046

Interval 40 (390000 steps performed)
10000/10000 [==============================] - 965s - reward: 0.0060   
88 episodes - episode_reward: 0.687 [0.412, 1.027] - loss: 0.008 - mean_absolute_error: 0.038 - mean_q: 1.678

Interval 41 (400000 steps performed)
10000/10000 [==============================] - 1023s - reward: 0.0059  
91 episodes - episode_reward: 0.650 [0.337, 1.051] - loss: 0.005 - mean_absolute_error: 0.030 - mean_q: 1.356

Interval 42 (410000 steps performed)
10000/10000 [==============================] - 946s - reward: 0.0058   
82 episodes - episode_reward: 0.707 [0.423, 0.82

In [16]:
# If TEST and no TOKEN, run some test experiments
train = False
env = RunEnv(visualize=True)
if not train:
    agent.load_weights(model)
    # Finally, evaluate our algorithm for 1 episode.
    agent.test(env, nb_episodes=1, visualize=True, nb_max_episode_steps=500)

Testing for 1 episodes ...
Episode 1: reward: 0.421, steps: 74
